In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import joblib
import os

# ==========================================
# 1. LOAD DATASETS
# ==========================================
print("--- Loading Data ---")
files = {
    'Emlakjet': 'data/raw/emlakjet/emlakjet_listings.xlsx',
    'Sahibinden': 'data/raw/sahibinden/sahibinden_enriched_listings.xlsx',
    'Hepsiemlak': 'data/raw/hepsiemlak/hepsiemlak_listings.xlsx'
}

dfs = []
for source, filename in files.items():
    if os.path.exists(filename):
        try: 
            print(f"Reading {filename}...")
            temp_df = pd.read_excel(filename)
            temp_df['Source'] = source
            dfs.append(temp_df)
        except Exception as e:
            print(f"Failed to read {filename} as Excel: {e}")
            try:
                temp_df = pd.read_csv(filename.replace('.xlsx', '.csv'))
                temp_df['Source'] = source
                dfs.append(temp_df)
                print(f"Read {filename} as CSV instead.")
            except:
                pass

if not dfs:
    raise FileNotFoundError("No data files found. Please check the 'data/raw' directory.")

df = pd.concat(dfs, ignore_index=True)
print(f"Total rows loaded: {len(df)}")

# ==========================================
# 2. DATA CLEANING & PREPROCESSING
# ==========================================
print("--- Cleaning Data ---")
def clean_price(price):
    if pd.isna(price): return np.nan
    if isinstance(price, (int, float)): return price
    price = str(price).replace('TL', '').replace('.', '').replace(',', '').strip()
    try:
        return float(price)
    except:
        return np.nan

def clean_numeric(value):
    if pd.isna(value): return np.nan
    if isinstance(value, (int, float)): return value
    value = str(value).replace('m²', '').replace('m2', '').replace('+1', '').strip()
    try:
        return float(value.split()[0])
    except:
        return np.nan

if 'Price' in df.columns:
    df['clean_price'] = df['Price'].apply(clean_price)
elif 'price' in df.columns:
    df['clean_price'] = df['price'].apply(clean_price)
else:
    raise ValueError("Could not find a 'Price' or 'price' column.")

df = df.dropna(subset=['clean_price'])

# Clean specific numeric features if they exist
for col in ['Area(m2)', 'Rooms', 'Building Age']:
    if col in df.columns:
        df[col] = df[col].apply(clean_numeric)

# Clean specific categorical features if they exist
for col in ['Furnishment', 'Listing Type']:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown').astype(str)

# Select Features
potential_numeric = ['Area(m2)', 'Rooms', 'Building Age']
potential_categorical = ['Furnishment', 'Listing Type']

numeric_features = [col for col in potential_numeric if col in df.columns]
categorical_features = [col for col in potential_categorical if col in df.columns]

print(f"Using Numeric Features: {numeric_features}")
print(f"Using Categorical Features: {categorical_features}")

X = df[numeric_features + categorical_features]
y = df['clean_price']

# Pipeline Setup
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# ==========================================
# 3. UNSUPERVISED LEARNING (Clustering)
# ==========================================
print("--- Running Unsupervised Learning ---")
try:
    X_processed = preprocessor.fit_transform(X)

    # PCA (Dimensionality Reduction for Visualization)
    n_components = min(2, X_processed.shape[1])
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_processed)

    if n_components >= 1: df['PCA1'] = X_pca[:, 0]
    if n_components >= 2: df['PCA2'] = X_pca[:, 1]
    else: df['PCA2'] = 0

    # K-Means (Market Segmentation)
    kmeans = KMeans(n_clusters=min(3, len(df)), random_state=42, n_init=10)
    df['Cluster'] = kmeans.fit_predict(X_processed)
except Exception as e:
    print(f"Skipping Unsupervised Learning: {e}")

# ==========================================
# 4. SUPERVISED LEARNING (Training)
# ==========================================
print("--- Running Supervised Learning ---")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42, max_depth=10),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
}

results = {}
best_model_score = -np.inf
best_model_name = ""
best_model_obj = None

for name, model in models.items():
    print(f"Training {name}...")
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', model)])
    
    # Train
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # Evaluate
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse) 
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred) if len(y_test) > 1 else 0.0

    results[name] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R2': r2}
    
    if r2 > best_model_score:
        best_model_score = r2
        best_model_name = name
        best_model_obj = clf

print(f"Best Model Selected: {best_model_name} (R2: {best_model_score:.4f})")
joblib.dump(best_model_obj, 'final_model.pkl')

# Generate Predictions for entire dataset to find deals
df['Predicted_Price'] = best_model_obj.predict(X)
df['Potential_Savings'] = df['Predicted_Price'] - df['clean_price']

# ==========================================
# 5. VISUALIZATION (Deal Finder Dashboard)
# ==========================================
print("--- Generating Deal Finder Results Image ---")
plt.style.use('seaborn-v0_8-whitegrid')
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle(f'Deal Finder Analysis Results (Best Model: {best_model_name})', fontsize=16)

# Plot 1: Model Comparison
names = list(results.keys())
values = [results[m]['R2'] for m in names]
colors = ['green' if n == best_model_name else 'gray' for n in names]
axes[0].bar(names, values, color=colors)
axes[0].set_title('Model Performance (R² Score)')
axes[0].set_ylim(0, 1.0)
axes[0].set_ylabel('R² Score (Higher is Better)')

# Plot 2: Clustering (Market Segments)
if 'PCA1' in df.columns and 'Cluster' in df.columns:
    sns.scatterplot(x='PCA1', y='PCA2', hue='Cluster', data=df, palette='viridis', ax=axes[1], s=100)
    axes[1].set_title('Market Segmentation (PCA + K-Means)')
    axes[1].set_xlabel('Principal Component 1')
    axes[1].set_ylabel('Principal Component 2')

# Plot 3: Actual vs Predicted (Deal Spotter)
# Deals are points ABOVE the identity line (Predicted > Actual)
sns.scatterplot(x='clean_price', y='Predicted_Price', data=df, ax=axes[2], alpha=0.6, label='Listings')
min_val = min(df['clean_price'].min(), df['Predicted_Price'].min())
max_val = max(df['clean_price'].max(), df['Predicted_Price'].max())
axes[2].plot([min_val, max_val], [min_val, max_val], 'r--', label='Fair Value (x=y)')

# Highlight Deals (Top 50 savings)
deals = df.nlargest(50, 'Potential_Savings')
axes[2].scatter(deals['clean_price'], deals['Predicted_Price'], color='green', s=50, label='Best Deals')

axes[2].set_title('Deal Finder: Actual vs. Predicted Price')
axes[2].set_xlabel('Actual Price (TL)')
axes[2].set_ylabel('Predicted Fair Price (TL)')
axes[2].legend()

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('data/outputs/deal_finder_results.png', dpi=300)
print("Saved visualization to 'data/outputs/deal_finder_results.png'")

# ==========================================
# 5b. ADDITIONAL VISUALIZATION (Distance Analysis)
# ==========================================
print("--- Generating Distance Analysis Image ---")
# Create additional visualization for distance analysis
fig2, axes2 = plt.subplots(1, 3, figsize=(18, 5))

# Distance correlations
if 'Distance to Metro (km)' in df.columns:
    # Clean distance column if needed (it might be string with 'km' or something, though load_all_data in analyze_data didn't seem to do much beyond basic cleaning. 
    # analyze_data.py didn't have specific cleaning for this, assuming it was numeric/clean enough or handled by pandas read_excel if numeric.
    # However, let's ensure it's numeric.
    pass 

has_plots = False

if 'Distance to Metro (km)' in df.columns:
    # Simple cleaning just in case
    df['Distance to Metro (km)'] = pd.to_numeric(df['Distance to Metro (km)'], errors='coerce')
    axes2[0].scatter(df['Distance to Metro (km)'], df['clean_price'], alpha=0.6, color='blue')
    axes2[0].set_title('Price vs Distance to Metro', fontsize=12, fontweight='bold')
    axes2[0].set_xlabel('Distance to Metro (km)')
    axes2[0].set_ylabel('Price (₺)')
    has_plots = True

if 'Distance to University (km)' in df.columns:
    df['Distance to University (km)'] = pd.to_numeric(df['Distance to University (km)'], errors='coerce')
    axes2[1].scatter(df['Distance to University (km)'], df['clean_price'], alpha=0.6, color='green')
    axes2[1].set_title('Price vs Distance to University', fontsize=12, fontweight='bold')
    axes2[1].set_xlabel('Distance to University (km)')
    axes2[1].set_ylabel('Price (₺)')
    has_plots = True

if 'Building Age' in df.columns:
    # Building Age is already cleaned in previous steps to be numeric
    axes2[2].scatter(df['Building Age'], df['clean_price'], alpha=0.6, color='orange')
    axes2[2].set_title('Price vs Building Age', fontsize=12, fontweight='bold')
    axes2[2].set_xlabel('Building Age (years)')
    axes2[2].set_ylabel('Price (₺)')
    has_plots = True

if has_plots:
    os.makedirs('visualizations', exist_ok=True)
    plt.tight_layout()
    plt.savefig('visualizations/distance_analysis.png', dpi=300, bbox_inches='tight')
    print("Saved visualization to 'visualizations/distance_analysis.png'")

# ==========================================
# 6. EFFICIENT EXCEL ORGANIZATION
# ==========================================
print("--- Saving Organized Excel File ---")
results_df = pd.DataFrame(results).T.reset_index().rename(columns={'index': 'Model'})

# Sort deals by highest savings first
best_deals_df = df[df['Potential_Savings'] > 0].sort_values(by='Potential_Savings', ascending=False)
best_deals_df = best_deals_df[['clean_price', 'Predicted_Price', 'Potential_Savings', 'Cluster'] + [col for col in df.columns if 'URL' in col or col == 'Source']]

with pd.ExcelWriter('data/outputs/ml_analysis_results.xlsx') as writer:
    # Sheet 1: Model Summary (High Level)
    results_df.to_excel(writer, sheet_name='1_Model_Performance', index=False)
    
    # Sheet 2: Actionable Insights (The Deals)
    best_deals_df.to_excel(writer, sheet_name='2_Best_Deals_Finder', index=False)
    
    # Sheet 3: Deep Dive (Clustering Data)
    cluster_cols = ['clean_price', 'Cluster', 'PCA1', 'PCA2']
    df[cluster_cols].to_excel(writer, sheet_name='3_Clustering_Analysis', index=False)

print("Saved organized results to 'data/outputs/ml_analysis_results.xlsx'")
print("\n" + "="*60)
print("ML ANALYSIS COMPLETE")
print("="*60)
